### Using Glue and PyDeequ to clean data 
AWS Glue automates the time consuming steps of data preparation for analytics. It lessens the effort to build, maintain, and produce an ETL job. Leveraging PyDeequ with Glue allows you to apply quantitative information to  anomalous data before transformation. 

**This tutorial builds upon the [Data Cleaning with AWS Glue](https://github.com/aws-samples/aws-glue-samples/blob/master/examples/data_cleaning_and_lambda.md) tutorial, and uses PyDeequ to analyze the data.** 

In [3]:
import sys
from awsglue.utils import getResolvedOptions

from awsglue.context import GlueContext
from pyspark.context import SparkContext

glueContext = GlueContext(SparkContext.getOrCreate())
session = glueContext.spark_session


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Extract - Load the dataset


We will be using a modified version of the Medicare-Provider payment data downloaded from two Data.CMS.gov sites: Inpatient Prospective Payment System Provider Summary for the Top 100 Diagnosis-Related Groups - FY2011, and Inpatient Charge Data FY 2011.

First let us look at the schema using a dataframe.

In [4]:
medicare = spark.read.format("com.databricks.spark.csv") \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .load('s3://awsglue-datasets/examples/medicare/Medicare_Hospital_Provider.csv')

medicare.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- DRG Definition: string (nullable = true)
 |-- Provider Id: string (nullable = true)
 |-- Provider Name: string (nullable = true)
 |-- Provider Street Address: string (nullable = true)
 |-- Provider City: string (nullable = true)
 |-- Provider State: string (nullable = true)
 |-- Provider Zip Code: integer (nullable = true)
 |-- Hospital Referral Region Description: string (nullable = true)
 |--  Total Discharges : integer (nullable = true)
 |--  Average Covered Charges : string (nullable = true)
 |--  Average Total Payments : string (nullable = true)
 |-- Average Medicare Payments: string (nullable = true)

### Let us use PyDeequ to analyze our data. 
By using the analyzer `DataType` we can see that the column "Provider Name" actually contains a string and integer datatype. Specifically, the results show only 2 string values and 163,063 Integral values. 

In [5]:
import pydeequ
from pydeequ.analyzers import *

analysisResult = AnalysisRunner(session) \
                    .onData(medicare) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(Completeness("Provider Name")) \
                    .addAnalyzer(DataType("Provider Id")) \
                    .run()
                    
analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show(n=13, truncate=False, vertical=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------+--------------------------+---------------------+
|entity |instance     |name                      |value                |
+-------+-------------+--------------------------+---------------------+
|Dataset|*            |Size                      |163065.0             |
|Column |Provider Name|Completeness              |1.0                  |
|Column |Provider Id  |Histogram.bins            |5.0                  |
|Column |Provider Id  |Histogram.abs.Boolean     |0.0                  |
|Column |Provider Id  |Histogram.ratio.Boolean   |0.0                  |
|Column |Provider Id  |Histogram.abs.Fractional  |0.0                  |
|Column |Provider Id  |Histogram.ratio.Fractional|0.0                  |
|Column |Provider Id  |Histogram.abs.Integral    |163063.0             |
|Column |Provider Id  |Histogram.ratio.Integral  |0.9999877349523196   |
|Column |Provider Id  |Histogram.abs.Unknown     |0.0                  |
|Column |Provider Id  |Histogram.ratio.Unknown   |0

### Load the datatable on a Glue Crawler
As a comparison between a dataFrame and dynamicFrame let us look at the `medicare` schema in a DynamicFrame. A dynamicFrame is similar to a dataframe, except that each record is self-describing, so no schema is initially required. By converting to a dynamicFrame we can see that the `provider id` column has two datatypes.  

In [6]:
from awsglue.dynamicframe import DynamicFrame

medicare_dynamicframe = glueContext.create_dynamic_frame.from_catalog(database = "medical_billing_demo", 
                                                                      table_name = "medicare_hospital_provider_csv")
medicare_dynamicframe.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- drg definition: string
|-- provider id: choice
|    |-- long
|    |-- string
|-- provider name: string
|-- provider street address: string
|-- provider city: string
|-- provider state: string
|-- provider zip code: long
|-- hospital referral region description: string
|-- total discharges: long
|-- average covered charges: string
|-- average total payments: string
|-- average medicare payments: string

To query the `provider id` column we need to resolve the choice. With DynamicFrames, we can use `resolveChoice` to cast `string` values into a `long` type. 

In [7]:
medicare_res = medicare_dynamicframe.resolveChoice(specs = [('provider id','cast:long')])
medicare_res.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- drg definition: string
|-- provider id: long
|-- provider name: string
|-- provider street address: string
|-- provider city: string
|-- provider state: string
|-- provider zip code: long
|-- hospital referral region description: string
|-- total discharges: long
|-- average covered charges: string
|-- average total payments: string
|-- average medicare payments: string

Where the `string` values cannot be cast AWS Glue replaced with a `Null`. Let us remove the anomalous datasets by using the `where` method and invoking an SQL query. 

In [8]:
medicare_dataframe = medicare_res.toDF()
medicare_dataframe = medicare_dataframe.where("`provider id` is NOT NULL")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 ## Transform
Next, let us make the data table easily accesible, to calculate our metrics. By viewing the average payments columns, we can see a leading dollar sign in the string values. 

In [9]:
medicare_dataframe.select("average covered charges", "average total payments", "average medicare payments").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+----------------------+-------------------------+
|average covered charges|average total payments|average medicare payments|
+-----------------------+----------------------+-------------------------+
|              $32963.07|              $5777.24|                 $4763.73|
|              $15131.85|              $5787.57|                 $4976.71|
|              $37560.37|              $5434.95|                 $4453.79|
|              $13998.28|              $5417.56|                 $4129.16|
|              $31633.27|              $5658.33|                 $4851.44|
|              $16920.79|              $6653.80|                 $5374.14|
|              $11977.13|              $5834.74|                 $4761.41|
|              $35841.09|              $8031.12|                 $5858.50|
|              $28523.39|              $6113.38|                 $5228.40|
|              $75233.38|              $5541.05|                 $4386.94|
|              $67327.92|

Remove the `$` character in the string by using user defined function, `udf`. Additionally rename the needed columns using the `withColumnRenamed` method to fit the python variable naming standard .

In [10]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

chop_f = udf(lambda x: x[1:], StringType())

medicare_dataframe = medicare_dataframe.withColumnRenamed("provider id", "provider_id") \
                    .withColumnRenamed("provider name", "provider_name") \
                    .withColumn("avg_covered_charges", chop_f(medicare_dataframe["average covered charges"]))\
                    .withColumn("avg_total_payments", chop_f(medicare_dataframe["average total payments"]))\
                    .withColumn("avg_medicare_payments", chop_f(medicare_dataframe["average medicare payments"]))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Using `cast`, change the payment columns from a `string` to a `float`. Use `select` to specify the specific columns needed for this tutorial. 

In [11]:
medicare_dataframe = medicare_dataframe.select("provider_id", "provider_name",
                                               medicare_dataframe.avg_covered_charges.cast("float"),
                                               medicare_dataframe.avg_total_payments.cast("float"),
                                               medicare_dataframe.avg_medicare_payments.cast("float"))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Great! We've utilized Glue to clean our data. 

Now let us analyze our payment columns using PyDeequ.


In [12]:
from pydeequ.analyzers import *

analysisResult = AnalysisRunner(session) \
                    .onData(medicare_dataframe) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(Mean("avg_covered_charges")) \
                    .addAnalyzer(Minimum("avg_medicare_payments")) \
                    .addAnalyzer(Maximum("avg_covered_charges")) \
                    .addAnalyzer(Maximum("avg_total_payments")) \
                    .run()
                    
analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(session, analysisResult)
analysisResult_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+-------+------------------+
| entity|            instance|   name|             value|
+-------+--------------------+-------+------------------+
| Column|avg_medicare_paym...|Minimum|1148.9000244140625|
| Column| avg_covered_charges|   Mean| 36134.25957028615|
| Column| avg_covered_charges|Maximum|        929118.875|
|Dataset|                   *|   Size|          163063.0|
| Column|  avg_total_payments|Maximum|       156158.1875|
+-------+--------------------+-------+------------------+

Using constraints on PyDeequ, we can conduct checks to further understand our large dataset. Let us check for 
constrints in our payment columns. Also, we can make sure that the datatype for `provider_id` is now solely numeric.  

In [13]:
from pydeequ.checks import *
from pydeequ.verification import *

check = Check(session, CheckLevel.Warning, "Medicare")

checkResult = VerificationSuite(session) \
    .onData(medicare_dataframe) \
    .addCheck(
        check.isComplete("provider_id")  \
        .hasDataType("provider_id", ConstrainableDataTypes.Numeric)
        .isNonNegative("avg_medicare_payments")) \
    .run()

print(f"Verification Run Status: {checkResult.status}")
checkResult_df = VerificationResult.checkResultsAsDataFrame(session, checkResult)
checkResult_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Verification Run Status: Success
+--------+-----------+------------+--------------------+-----------------+------------------+
|   check|check_level|check_status|          constraint|constraint_status|constraint_message|
+--------+-----------+------------+--------------------+-----------------+------------------+
|Medicare|    Warning|     Success|CompletenessConst...|          Success|                  |
|Medicare|    Warning|     Success|AnalysisBasedCons...|          Success|                  |
|Medicare|    Warning|     Success|ComplianceConstra...|          Success|                  |
+--------+-----------+------------+--------------------+-----------------+------------------+